In [ ]:
import csv
import sys
import os
import threading
import unicodedata
import string
import argparse
from collections import Counter
from math import ceil

# ==============================================================================
# 1. CLASSES E MODELAGEM
# ==============================================================================
class Trabalho: # -> Criação da classe trabalho, que é o coração do código
    def __init__(self, nm_programa, nm_grande_area, nm_orientador, nm_area, nm_producao):
        self.programa = nm_programa
        self.grande_area = nm_grande_area
        self.orientador = nm_orientador
        self.area = nm_area
        self.titulo = nm_producao
        # -> Colunas relevantes ditas na folha de questão
    def __repr__(self):
        return f"<Trabalho: {self.titulo[:20]}...>"
#  -> A classe cria objetos trabalho, asssim como expresso na folha de questões

# ==============================================================================
# 2. FUNÇÕES PURAS E UTILITÁRIAS
# ==============================================================================

# Função que irá carregar o stopwords(palavras chaves a serem removidas ou alteradas)
def carregar_stopwords(caminhos):
    """Carrega stopwords de múltiplos arquivos."""
    stops = set() # -> Criação de um conjunto para facilitar o armazenamento e uso de funções de agregação como "IN"

    for caminho in caminhos: # -> Um loop para leitura dos arquivos stopwords caminhos -> arquivo(s) caminho -> Arquivo-fonte
        if os.path.exists(caminho):
            with open(caminho, 'r', encoding='utf-8') as f: # Abertura e leitura do arquivo, com f sendo a variável que pasará lendo o arquivo
                stops.update(line.strip().lower() for line in f)
                # Update no conjunto stops, adicionando palavras baseado em espaços entres as mesmas, deixando tudo em lower. para cada linha da variável "f" que
                # passa lendo os stops
        else:
            print(f"Aviso: Arquivo de stopwords '{caminho}' não encontrado.")
    return stops
# Aqui é uma função que irá normalizar as palavras encontradas e armazenadas em uma lista anteriormente definida de palavras
def normalizar_token(token):
    """
    Aplica normalização: minúsculas, remove pontuação, remove acentos.

    """
    # 1. Minúsculas
    token = token.lower()

    # 2. Remove pontuação
    token = token.translate(str.maketrans('', '', string.punctuation))

    # 3. Remove acentos (Normalização Unicode)
    nfkd_form = unicodedata.normalize('NFKD', token)
    token = "".join([c for c in nfkd_form if not unicodedata.combining(c)])

    return token
# Leitura do Arquivo com nossos dados, lendo o arquivo e categorizando as coluna chave que serão usadas em toda lógica de visualização das posteriores informações
def ler_dataset(caminho_arquivo):
    """Lê o CSV e retorna lista de objetos Trabalho. """
    trabalhos = []
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            # O delimitador do arquivo enviado é ';'
            leitor = csv.DictReader(f, delimiter=';')
            for row in leitor:
                # Verifica se as chaves existem para evitar KeyError em arquivos malformados
                if 'NM_PROGRAMA' in row:
                    trabalhos.append(Trabalho(
                        nm_programa=row.get('NM_PROGRAMA', ''),
                        nm_grande_area=row.get('NM_GRANDE_AREA_CONHECIMENTO', ''),
                        nm_orientador=row.get('NM_ORIENTADOR', ''),
                        nm_area=row.get('NM_AREA_CONHECIMENTO', ''),
                        nm_producao=row.get('NM_PRODUCAO', '')
                    ))
    except FileNotFoundError:
        print(f"Erro: Arquivo {caminho_arquivo} não encontrado.")
        sys.exit(1)
    except Exception as e:
        print(f"Erro ao ler dataset: {e}")
        sys.exit(1)
    return trabalhos
# A função que irá gerar os rankings propostos na atividade
def gerar_ranking_metadados(lista_objetos, extrator_chave, top_n=10):
    """
    Função genérica para rankings simples.

    """
    # Programação Funcional: Map para extrair apenas o campo desejado
    chaves = map(extrator_chave, lista_objetos)

    # Agregação
    contador = Counter(chaves)

    # Retorna os Top N
    return contador.most_common(top_n)

# ==============================================================================
# 3. PROGRAMAÇÃO CONCORRENTE: CONTAGEM DE PALAVRAS
# ==============================================================================
# Criação da primeira classe relacionda ao uso de thread -> Essa classe definirá o processamento das atividades que serma feitas por cada thread
class WordCounterWorker(threading.Thread):
    """Thread responsável por processar um bloco de trabalhos."""

    def __init__(self, trabalhos_chunk, stopwords):
        super().__init__()
        self.trabalhos = trabalhos_chunk # A thread receberá uma parte dos trabalhos
        self.stopwords = stopwords # As palvaras a serem usadas na filtragem (Palavras para serem removidas da contagem)
        self.resultado_parcial = {} # Um mini dicionário que irá armazenar momentaneamente a contagem de palavras por thread, antes de ser colocada em um cojunto global

    def run(self):
        """Executa o processamento do bloco."""
        local_counter = {}

        for trabalho in self.trabalhos:
            # Tokenização simples por espaço
            palavras_cruas = trabalho.titulo.split()

            for palavra in palavras_cruas:
                # Normalização
                token = normalizar_token(palavra) # Uso na função de normalização de palavras feita lá em cima

                # Filtros
                # Ignorar <= 3 chars e stopwords
                if len(token) <= 3:
                    continue
                if token in self.stopwords:
                    continue

                # Contagem manual (proibido lib de contagem aqui)
                if token in local_counter:
                    local_counter[token] += 1
                else:
                    local_counter[token] = 1

        self.resultado_parcial = local_counter # O contador local joga as informaçõe processadas no dicinonário temporário criado

# O método que organiza os trabalhos para as threads criadas
def executar_contagem_concorrente(trabalhos, stopwords, n_threads=4):
    """
    Orquestrador das threads. Divide dados e agrega resultados.

    """
    tamanho_bloco = ceil(len(trabalhos) / n_threads) # definição dos blocos de contagem para cada thread. Sempre arredondando para cima
    threads = []

    # 1. Divisão e Início das Threads
    for i in range(n_threads):
        inicio = i * tamanho_bloco
        fim = inicio + tamanho_bloco
        chunk = trabalhos[inicio:fim]
        # Definição do tamanho da chunk

        if not chunk: continue

        worker = WordCounterWorker(chunk, stopwords) # criação dos trabalhadores com base na classe que indica suas tarefas. Recebe um pedaço do bloco de palavras e as stopw como arg
        threads.append(worker) # Adiciona os workers no dicionário de threads
        worker.start() # Começa o processo com os workers

    # 2. Aguarda término (Join)

    for t in threads:
        t.join()

    # 3. Agregação Global (Main Thread)
    contagem_global = {}
    for t in threads:
        parcial = t.resultado_parcial
        for palavra, qtd in parcial.items():
            if palavra in contagem_global:
                contagem_global[palavra] += qtd
            else:
                contagem_global[palavra] = qtd
      # Contador global é criado, para receber as palavras que virão dos contadores parciais de cada thread
      # Aqui, por meio de loops aninhados, percorremos as threads, e percorremos a quantidade de palavras que existem nos contadores parciais de cada thread
      # Realiza a contagem das palavras de forma manual. Se a palavra está no cont global, adiciona ela novamente, caso não, adiciona ela

    # 4. Ordenação Final para Ranking (Top 20)
    ranking = sorted(contagem_global.items(), key=lambda x: x[1], reverse=True)[:20]

    return ranking

# ==============================================================================
# 4. MAIN PIPELINE
# ==============================================================================
def main():
    # 1. Parsing de Argumentos
    parser = argparse.ArgumentParser(description="Pipeline AP2 CAPES")
    parser.add_argument('--dataset', type=str, default='/content/drive/MyDrive/py/ap2-capes-ufc-2021.csv', help='Caminho do CSV')
    args, unknown = parser.parse_known_args() # Modified line

    print(f"Iniciando processamento do arquivo: {args.dataset}...")

    # 2. Carregar Stopwords (Assume arquivos na mesma pasta)
    stopwords = carregar_stopwords(['/content/drive/MyDrive/py/stopwords_en.txt', '/content/drive/MyDrive/py/stopwords_pt.txt'])

    # 3. Leitura do Dataset
    lista_trabalhos = ler_dataset(args.dataset)
    print(f"Total de trabalhos carregados: {len(lista_trabalhos)}")
    print("-" * 50)

    # 4. Rankings Simples (Map/Filter/Reduce logic via Counter)

    # Top 10 Programas
    rank_programas = gerar_ranking_metadados(lista_trabalhos, lambda t: t.programa, top_n=10)
    print("### TOP 10 PROGRAMAS ###")
    for i, (prog, qtd) in enumerate(rank_programas, 1):
        print(f"{i}. {prog}: {qtd}")
    print("-" * 50)

    # Top 10 Orientadores
    rank_orientadores = gerar_ranking_metadados(lista_trabalhos, lambda t: t.orientador, top_n=10)
    print("### TOP 10 ORIENTADORES ###")
    for i, (ori, qtd) in enumerate(rank_orientadores, 1):
        print(f"{i}. {ori}: {qtd}")
    print("-" * 50)

    # Top 10 Áreas (Grande Área -> Área)
    rank_areas = gerar_ranking_metadados(
        lista_trabalhos,
        lambda t: f"{t.grande_area} -> {t.area}",
        top_n=10
    )
    print("### TOP 10 ÁREAS (Grande Área -> Área) ###")
    for i, (area, qtd) in enumerate(rank_areas, 1):
        print(f"{i}. {area}: {qtd}")
    print("-" * 50)

    # 5. Ranking de Palavras (Concorrente)
    print("### TOP 20 PALAVRAS EM TÍTULOS (Threading) ###")
    rank_palavras = executar_contagem_concorrente(lista_trabalhos, stopwords, n_threads=4)

    print(f"{'#':<4} {'PALAVRA':<20} {'FREQ':<5}")
    for i, (palavra, freq) in enumerate(rank_palavras, 1):
        print(f"{i:<4} {palavra:<20} {freq:<5}")

if __name__ == "__main__":
    main()

Iniciando processamento do arquivo: /content/drive/MyDrive/py/ap2-capes-ufc-2021.csv...
Total de trabalhos carregados: 1287
--------------------------------------------------
### TOP 10 PROGRAMAS ###
1. ECONOMIA: 61
2. EDUCAÇÃO: 60
3. LETRAS: 58
4. QUÍMICA: 42
5. DIREITO: 39
6. LINGÜÍSTICA: 33
7. ADMINISTRAÇÃO E CONTROLADORIA: 32
8. FÍSICA: 30
9. PSICOLOGIA: 30
10. ENGENHARIA DE TELEINFORMÁTICA: 30
--------------------------------------------------
### TOP 10 ORIENTADORES ###
1. SERGIO AQUINO DE SOUZA: 10
2. WAGNER BANDEIRA ANDRIOLA: 7
3. MAGNO JOSE DUARTE CANDIDO: 6
4. ALESSANDRA CARVALHO DE VASCONCELOS: 6
5. JOAO MÁRIO SANTOS DE FRANCA: 6
6. MARTA MARIA DE FRANCA FONTELES: 6
7. JOSE GERARDO VASCONCELOS: 6
8. BARTOLOMEU WARLENE SILVA DE SOUZA: 5
9. FERNANDO LUIZ MARCELO ANTUNES: 5
10. BRENO MAGALHAES FREITAS: 5
--------------------------------------------------
### TOP 10 ÁREAS (Grande Área -> Área) ###
1. CIÊNCIAS SOCIAIS APLICADAS -> ECONOMIA: 68
2. LINGÜÍSTICA, LETRAS E ARTES -> LE